<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SRA-metadata-download" data-toc-modified-id="SRA-metadata-download-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SRA metadata download</a></span><ul class="toc-item"><li><span><a href="#Import-needed-packages" data-toc-modified-id="Import-needed-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import needed packages</a></span></li><li><span><a href="#Download-per-sample-metadata-from-NCBI-FTP-using-previously-parsed-for-now,-will-need-to-write-code-to-parse" data-toc-modified-id="Download-per-sample-metadata-from-NCBI-FTP-using-previously-parsed-for-now,-will-need-to-write-code-to-parse-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Download per sample metadata from <a href="https://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata/" target="_blank">NCBI FTP</a> <font color="red">using previously parsed for now, will need to write code to parse</font></a></span></li><li><span><a href="#Sample-and-study-technical-metadata-need-to-find-where-this-is-hiding" data-toc-modified-id="Sample-and-study-technical-metadata-need-to-find-where-this-is-hiding-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Sample and study technical metadata <font color="red">need to find where this is hiding</font></a></span></li><li><span><a href="#Download-BioSample-attribute-definitions" data-toc-modified-id="Download-BioSample-attribute-definitions-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Download BioSample <a href="https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/" target="_blank">attribute definitions</a></a></span></li></ul></li></ul></div>

# SRA metadata download
Adam Klie<br>11/23/2019<br>
Script to download relevant data for building, training and testing metadata prediction models

## Import needed packages

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import ipywidgets as widgets
from IPython.display import IFrame
import qgrid

## Download per sample metadata from [NCBI FTP](https://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata/) <font color='red'>using previously parsed for now, will need to write code to parse</font>

In [2]:
IFrame('https://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata/', width=800, height=450)

In [3]:
#!wget -O ../data/NCBI_SRA_Metadata_20181202.tar.gz https://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata/NCBI_SRA_Metadata_20181202.tar.gz

In [4]:
#!tar -vxzf ../data/NCBI_SRA_Metadata_20181202.tar.gz
#sra_raw = pd.read_json('../data/NCBI_SRA_Metadata_20181202')

## Sample and study technical metadata <font color='red'>need to find where this is hiding</font>

## Download BioSample [attribute definitions](https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/)

In [6]:
IFrame('https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/', width=800, height=500)

In [7]:
#!wget -O ../data/BioSampleAttributes.xml https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/?format=xml

In [10]:
def parseXML(xmlfile):
  
    # create element tree object
    tree = ET.parse(xmlfile)
  
    # get root element
    root = tree.getroot()
    
    # create empty list for news items
    attribute_df = pd.DataFrame()
  
    # iterate news items
    for item in root.findall('./Attribute'):
        
        # empty news dictionary
        attribute = {}
  
        # iterate child elements of item
        for child in item:
            attribute[child.tag] = child.text
            
        # append news dictionary to news items list
        attribute_df = attribute_df.append(attribute, ignore_index = True)
      
    # return news items list
    return attribute_df

In [11]:
df = parseXML('../data/BioSampleAttributes.xml').set_index('Name')

In [12]:
qgrid_widget = qgrid.show_grid(df, show_toolbar=True)

In [13]:
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [16]:
df.to_pickle('../data/BioSampleAttributes.pickle')